In [32]:
from langchain_groq import ChatGroq
 

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Who was the first person to go to the moon?")
print(response.content)



The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.

However, it's worth noting that Armstrong was not alone on the mission. He was accompanied by fellow astronauts Edwin "Buzz" Aldrin and Michael Collins. While Armstrong was the first to walk on the moon, Aldrin joined him on the surface shortly afterwards. Collins remained in orbit around the moon in the command module Columbia.

Neil Armstrong's historic moonwalk marked a major milestone in space exploration and paved the way for future human spaceflight missions.


In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/110690555461018310")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Engineer III, Infrastructure, Core — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results1,994  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Gurgaon, Haryana, India; +2 more; +1 moreSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, AI/ML GenAI, Google Cloud AISunnyvale, CA, USA; Kirkland, WA, USASoftware Engineer, Silicon Engineering ProductivityBengaluru, Karnataka, IndiaSenior Software Engineer, Search UI and AI InfrastructureSão Paulo, State of São Paulo, BrazilCloud Technical Solutions Engineer, Google Workspace (English, Japanese)Tokyo, JapanTechnical Program Manager II, Infrastructure Resource M

In [27]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    the scraped text is from career's page of a website.
    your job is to extract the job postings and return them in JSON format containing the following keys:
    'skills', 'roles','experience' and 'description'.
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
type(res.content)



str

In [30]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'skills': ['Software development',
   'Data structures',
   'Algorithms',
   'Distributed systems',
   'Networking',
   'Data storage',
   'Security',
   'Artificial intelligence',
   'Natural language processing',
   'UI design',
   'Mobile'],
  'roles': 'Software Engineer III, Infrastructure, Core',
  'experience': '2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree. 2 years of experience with data structures or algorithms in either an academic or industry setting.',
  'description': "Google's software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact with information and one another. Our products need to handle information at massive scale, and extend well beyond web search."},
 {'skills': ['Sales', 'Customer solutions', 'Mid-market sales'],
  'roles': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
  'experience': '

In [33]:
type(json_res)

list

In [34]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [35]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [37]:

job = {
    "skills": ["Python", "Machine Learning", "Data Analysis"]
}


links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
print(links)


[[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/ios-portfolio'}]]


In [38]:
job

{'skills': ['Python', 'Machine Learning', 'Data Analysis']}

In [40]:
job = json_res[0]  
print(job['skills'])  


['Software development', 'Data structures', 'Algorithms', 'Distributed systems', 'Networking', 'Data storage', 'Security', 'Artificial intelligence', 'Natural language processing', 'UI design', 'Mobile']


In [42]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Swarnadipta, a business development executive at Accenture. Accenture is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Accenture 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Accenture's portfolio: {link_list}
        Remember you are Swarnadipta, BDE at Accenture. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Empowering Google's Next-Generation Technologies with Accenture's Expertise

Dear Hiring Manager,

I came across the job description for a Software Engineer III, Infrastructure, Core at Google, and I am excited to introduce Accenture's capabilities in fulfilling your needs. With our extensive experience in software development, data structures, algorithms, and distributed systems, I believe we can make a significant impact in supporting Google's mission to develop next-generation technologies.

At Accenture, we have a proven track record of empowering numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has hands-on experience in developing innovative solutions using various programming languages, data storage, security, artificial intelligence, and natural language processing.

I would like to highlight some of our relevant portfolio pieces that demonstrate our capabil